In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq
from datasets import load_metric
import numpy as np
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer


In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [7]:

model_checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, src_lang='kor_Hang', tgt_lang='eng_Latn')
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids('eng_Latn')


In [8]:
dataset = load_dataset('json', data_files='./일상생활및구어체_영한_train_set.json', field='data')

split_datasets = dataset["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

# declare preprocessing code 
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['ko']
    targets = examples['en']

    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_input_length, truncation=True)

    return model_inputs            


Found cached dataset json (/home/juhwan/.cache/huggingface/datasets/json/default-495f2ce9a157e059/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [00:00<00:00, 25.53it/s]
Loading cached split indices for dataset at /home/juhwan/.cache/huggingface/datasets/json/default-495f2ce9a157e059/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-9703b2b7b7ffbf1d.arrow and /home/juhwan/.cache/huggingface/datasets/json/default-495f2ce9a157e059/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-a1f820f7c9740879.arrow


In [9]:
# dataset
from datasets import load_dataset
dataset = load_dataset('json', data_files='./일상생활및구어체_영한_train_set.json', field='data')

split_datasets = dataset["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

# declare preprocessing code 

max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['ko']
    targets = examples['en']

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs            



tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Found cached dataset json (/home/juhwan/.cache/huggingface/datasets/json/default-495f2ce9a157e059/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [00:00<00:00, 25.63it/s]
Loading cached split indices for dataset at /home/juhwan/.cache/huggingface/datasets/json/default-495f2ce9a157e059/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-9703b2b7b7ffbf1d.arrow and /home/juhwan/.cache/huggingface/datasets/json/default-495f2ce9a157e059/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-a1f820f7c9740879.arrow
Map:   0%|          | 0/1080276 [00:00<?, ? examples/s]/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as y

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
metric = load_metric("bleu")

In [ ]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    print(decoded_preds, decoded_labels)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result



In [ ]:

args = Seq2SeqTrainingArguments(
    f"nllb-finetuned-aihub-ko-to-en",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,

)


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
# 학습하기전 초기모델 점수
trainer.evaluate(max_length=max_target_length)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(max_length=max_target_length)